In [1]:
import rdflib
from rdflib import URIRef
from rdflib.graph import ConjunctiveGraph
g = ConjunctiveGraph()

In [2]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, XML, JSON
from rdflib import Graph

In [3]:
def get_sparql_dataframe(service, query):
    #Helper function to convert SPARQL results into a pandas data frame
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()
    
    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    
    out =[]
    for row in processed_results['results']['bindings']:
        item =[]
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

In [4]:
wds = "http://dbpedia.org/sparql"

In [5]:
rq =""" 
PREFIX  dbpedia-owl:  <http://dbpedia.org/ontology/>
PREFIX dbpedia: <http://dbpedia.org/resource>
PREFIX dbpprop: <http://dbpedia.org/property>
PREFIX cat: <http://dbpedia.org/resource/Category:>
PREFIX dbp: <http://dbpedia.org/property/owner>

SELECT  DISTINCT ?company_name ?Key_Person ?product ?type_ck
WHERE {
   ?company dct:subject cat:Retail_companies_of_the_United_Kingdom.
   ?company dct:subject ?subject.
   ?company rdfs:label ?company_name.
   ?company dbo:keyPerson ?keyPerson.
   ?keyPerson rdfs:label ?Key_Person.
   ?company dbo:product ?dboproduct.
   ?dboproduct rdfs:label ?product.
   ?company dbo:type ?type.
   ?type rdfs:label ?type_ck.
   FILTER ( lang(?company_name) = 'en' and lang(?Key_Person) = 'en' and lang(?product) = 'en' and lang(?type_ck) = 'en')
}

GROUP BY ?company
"""

In [6]:
df = get_sparql_dataframe(wds,rq)

In [7]:
df

,company_name,Key_Person,product,type_ck
0,Tesco,Dave Lewis (businessman),Big-box store,Public limited company
1,JD Sports,Chairman,Sportswear (activewear),Public limited company
2,Morrisons,Ken Morrison,Supermarket,Public limited company
3,Carphone Warehouse,Charles Dunstone,Service (economics),Private limited company
4,Carphone Warehouse,Chief Executive,Landline phone,Private limited company
5,Matalan,John Hargreaves (businessman),Clothing,Privately held company
6,Topshop,Ian Grabiner,Shoe,Private company limited by shares
7,Topshop,Chief executive officer,Clothing,Private company limited by shares
8,Tesco International operations,Richard Broadbent,Supermarket,Public limited company
9,Tesco International operations,Dave Lewis (Tesco),Cash and carry (wholesale),Public limited company


In [8]:
chandu_mail = df.to_csv("chandu_mail.csv")

In [9]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

In [10]:
import os
import os.path
import sys
import json

In [11]:
cwd = os.getcwd()

In [12]:
cwd

'C:\\Users\\subro\\Documents\\KG'

In [13]:
files = os.listdir("C:\\Users\\subro\\Documents\\KG\\final")
files

['answer_scores.csv',
 'chandu_mail.csv',
 'emails.csv',
 'groups.csv',
 'group_memberships.csv',
 'matches.csv',
 'professionals.csv',
 'questions.csv',
 'question_scores.csv',
 'school_memberships.csv',
 'students.csv',
 'tags.csv',
 'tag_questions.csv',
 'tag_users.csv']

In [14]:
data_parts = {}
for file_name in files:
    file_id = file_name.split(".")[0]
    data_parts[file_id] = pd.read_csv("C:\\Users\\subro\\Documents\\KG\\final\\" + file_name)
    print(file_id)
    print(data_parts[file_id].shape)

answer_scores
(51138, 2)
chandu_mail
(137, 5)
emails
(1850101, 4)
groups
(49, 2)
group_memberships
(1038, 2)
matches
(4316275, 2)
professionals
(28152, 5)
questions
(23931, 5)
question_scores
(23928, 2)
school_memberships
(5638, 2)
students
(30971, 3)
tags
(16269, 2)
tag_questions
(76553, 2)
tag_users
(136663, 2)


In [15]:
def add_nodes(G, df, col, type_name):
    """Add entities to G from the 'col' column of the 'df' DataFrame. The new nodes are annotated with 'type_name' label."""
    nodes = list(df[~df[col].isnull()][col].unique())
    G.add_nodes_from([(n,dict(type=type_name)) for n in nodes])
    print("Nodes (%s,%s) were added" % (col, type_name))
    
def add_links(G, df, col1, col2, type_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[(~df[col1].isnull()) & (~df[col2].isnull())]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    G.add_edges_from([(src, trg, dict(type=type_name)) for src, trg in links])
    print("Edges (%s->%s,%s) were added" % (col1, col2, type_name))

In [16]:
G = nx.DiGraph()

In [17]:
school_memberships.csv: Just like group_memberships, but for schools instead.

students.csv: Students are the most important people on CareerVillage.org. They tend to range in age from about 14 to 24. They're all over the world, and they're the reason we exist!

tag_questions.csv: Every question can be hashtagged. We track the hashtag-to-question pairings, and put them into this file.

tag_users.csv: Users of any type can follow a hashtag. This shows you which hashtags each user follows.

tags.csv: Each tag gets a name.

question_scores.csv: "Hearts" scores for each question.

answer_scores.csv: "Hearts" scores for each answer.

SyntaxError: invalid syntax (<ipython-input-17-be34f5ed860a>, line 1)

In [18]:
#add_nodes(G, data_parts["answers"], "answers_id", "answer")
#add_nodes(G, data_parts["comments"], "comments_id", "comment")
add_nodes(G, data_parts["groups"], "groups_id", "group")
add_nodes(G, data_parts["groups"], "groups_group_type", "group_type")
add_nodes(G, data_parts["professionals"], "professionals_id", "professional")
add_nodes(G, data_parts["professionals"], "professionals_industry", "industry")
add_nodes(G, data_parts["questions"], "questions_id", "question")
add_nodes(G, data_parts["school_memberships"], "school_memberships_school_id", "school")
add_nodes(G, data_parts["students"], "students_id", "student")
add_nodes(G, data_parts["tags"], "tags_tag_id", "tag")

Nodes (groups_id,group) were added
Nodes (groups_group_type,group_type) were added
Nodes (professionals_id,professional) were added
Nodes (professionals_industry,industry) were added
Nodes (questions_id,question) were added
Nodes (school_memberships_school_id,school) were added
Nodes (students_id,student) were added
Nodes (tags_tag_id,tag) were added


In [19]:
#add_links(G, data_parts["answers"], "answers_id", "answers_question_id", "question")
#add_links(G, data_parts["answers"], "answers_id", "answers_author_id", "author")
#add_links(G, data_parts["comments"], "comments_id", "comments_parent_content_id", "parent_content")
#add_links(G, data_parts["comments"], "comments_id", "comments_author_id", "author")
add_links(G, data_parts["group_memberships"], "group_memberships_user_id", "group_memberships_group_id", "member")
add_links(G, data_parts["groups"], "groups_id", "groups_group_type", "type")
add_links(G, data_parts["professionals"], "professionals_id", "professionals_industry", "type")
add_links(G, data_parts["questions"], "questions_id", "questions_author_id", "author")
add_links(G, data_parts["school_memberships"], "school_memberships_user_id", "school_memberships_school_id", "member")
add_links(G, data_parts["tag_questions"], "tag_questions_question_id", "tag_questions_tag_id", "tag")
add_links(G, data_parts["tag_users"], "tag_users_user_id", "tag_users_tag_id", "follow")

Edges (group_memberships_user_id->group_memberships_group_id,member) were added
Edges (groups_id->groups_group_type,type) were added
Edges (professionals_id->professionals_industry,type) were added
Edges (questions_id->questions_author_id,author) were added
Edges (school_memberships_user_id->school_memberships_school_id,member) were added
Edges (tag_questions_question_id->tag_questions_tag_id,tag) were added
Edges (tag_users_user_id->tag_users_tag_id,follow) were added


In [20]:
students = data_parts["students"]
profs = data_parts["professionals"]
students = students[~students["students_location"].isnull()]
profs = profs[~profs["professionals_location"].isnull()]

In [21]:
students.head(10)

,students_id,students_location,students_date_joined
59,9658267bc2564a85bad1e802de5fb597,"Wayne, Pennsylvania",2012-01-01 05:00:00 UTC+0000
78,d49795757f274dd690bdce3936edea20,"New York, New York",2012-01-01 05:00:00 UTC+0000
97,fae3c431698a418882c4204e5f3ee244,"Lakewood, Ohio",2012-01-01 05:00:00 UTC+0000
104,2e67080928c74a4d863b2bcc60921152,"New York, New York",2012-01-01 05:00:00 UTC+0000
111,2e7d1b7f1d474530b8114ab72686726f,"London, Ontario, Canada",2012-01-01 05:00:00 UTC+0000
124,94f41c4228e2452aa8ddf786d02b06c0,"Burlingame, California",2012-01-01 05:00:00 UTC+0000
134,da1239a179894367a43dfa56c8ef6515,"San Francisco, California",2012-01-04 05:00:00 UTC+0000
136,b3e4125677e84e77922a562686d8e161,"Tempe, Arizona",2012-01-04 05:00:00 UTC+0000
142,8cd865a6e8ce41d39095bc79340ab352,"Boston, Massachusetts",2012-01-04 05:00:00 UTC+0000
147,16908136951a48ed942738822cedd5c2,"Palm Harbor, Florida",2012-01-04 05:00:00 UTC+0000


In [22]:
profs.head(10)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",NaN,NaN,2011-11-09 20:39:29 UTC+0000
10,7d425e8d7cfb4fe7b0702fff4d6d84e7,"Milwaukee, Wisconsin",NaN,Ph.D. Student in Political Science at Harvard ...,2011-12-26 05:00:00 UTC+0000
15,68ecc66323b8418092fdd724aaf5be94,"New York, New York",Special Events,Event Hospitality & Operations Specialist,2011-12-26 05:00:00 UTC+0000
18,971937860c164896a0771bf502bad482,"Wilmington, Vermont",NaN,NaN,2011-12-26 05:00:00 UTC+0000
19,0c9a2748560541be9fe2df0d7be88282,"New York, New York",NaN,NaN,2011-12-26 05:00:00 UTC+0000
27,2a8328f6ceb548699a64293f986f3d19,"Washington, Washington",NaN,NaN,2011-12-26 05:00:00 UTC+0000
28,8e75aca48b434c79b8b4729ef871cb50,"Boston, Massachusetts",NaN,NaN,2011-12-26 05:00:00 UTC+0000
29,01b3e033848b41f6b7a55cefc59ba61a,"San Francisco, California",NaN,NaN,2011-12-26 05:00:00 UTC+0000
30,c79d4e4fd9af4ab7a7e6b6f433128476,"New York, New York","education, tech, media","media, technology, education",2011-12-26 05:00:00 UTC+0000


In [23]:
locs1 = list(students["students_location"])
locs2 = list(profs["professionals_location"])
locs = [loc.lower() for loc in locs1+locs2]
locs_unique = list(set(locs))

In [24]:
cnt = Counter(locs)
cnt.most_common()[:10]

[('new york, new york', 2650),
 ('bengaluru, karnataka, india', 1284),
 ('los angeles, california', 1280),
 ('boston, massachusetts', 1271),
 ('houston, texas', 1032),
 ('san francisco, california', 975),
 ('chicago, illinois', 920),
 ('california, california', 894),
 ('greater new york city area', 745),
 ('atlanta, georgia', 738)]

In [25]:
new_edges = []
new_nodes = []
for loc in locs_unique:
    loc_hierarchy = loc.split(", ")
    loc_nodes = [] # due to city name duplicates in the world
    k = len(loc_hierarchy)
    for i in range(k):
        loc_nodes.append('_'.join(loc_hierarchy[i:]))
    new_nodes += loc_nodes
    loc_links = [(loc_nodes[i],loc_nodes[i+1], dict(type="location"))  for i in range(k-1)]
    new_edges += loc_links
new_nodes = list(set(new_nodes))
new_nodes = [(n, dict(type="location")) for n in new_nodes]

In [26]:
G.add_nodes_from(new_nodes)
G.add_edges_from(new_edges)
print(len(new_edges), len(new_nodes))

7253 7160


In [27]:
list(G.in_edges("united kingdom"))[:5]

[('england_united kingdom', 'united kingdom'),
 ('sheffield_united kingdom', 'united kingdom'),
 ('scotland_united kingdom', 'united kingdom'),
 ('portsmouth_united kingdom', 'united kingdom'),
 ('northern ireland_united kingdom', 'united kingdom')]

In [28]:
list(G.in_edges("england_united kingdom"))[:5]

[('henley-on-thames_england_united kingdom', 'england_united kingdom'),
 ('glastonbury_england_united kingdom', 'england_united kingdom'),
 ('salisbury_england_united kingdom', 'england_united kingdom'),
 ('hull_england_united kingdom', 'england_united kingdom'),
 ('huntingdon_england_united kingdom', 'england_united kingdom')]

In [29]:
students["students_location"] = students["students_location"].apply(lambda x: "_".join(x.lower().split(", ")))
profs["professionals_location"] = profs["professionals_location"].apply(lambda x: "_".join(x.lower().split(", ")))

C:\Users\subro\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\subro\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
students.head(8)

,students_id,students_location,students_date_joined
59,9658267bc2564a85bad1e802de5fb597,wayne_pennsylvania,2012-01-01 05:00:00 UTC+0000
78,d49795757f274dd690bdce3936edea20,new york_new york,2012-01-01 05:00:00 UTC+0000
97,fae3c431698a418882c4204e5f3ee244,lakewood_ohio,2012-01-01 05:00:00 UTC+0000
104,2e67080928c74a4d863b2bcc60921152,new york_new york,2012-01-01 05:00:00 UTC+0000
111,2e7d1b7f1d474530b8114ab72686726f,london_ontario_canada,2012-01-01 05:00:00 UTC+0000
124,94f41c4228e2452aa8ddf786d02b06c0,burlingame_california,2012-01-01 05:00:00 UTC+0000
134,da1239a179894367a43dfa56c8ef6515,san francisco_california,2012-01-04 05:00:00 UTC+0000
136,b3e4125677e84e77922a562686d8e161,tempe_arizona,2012-01-04 05:00:00 UTC+0000


In [31]:
profs.head(10)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
2,0c673e046d824ec0ad0ebe012a0673e4,new york_new york,NaN,NaN,2011-10-18 17:31:26 UTC+0000
3,977428d851b24183b223be0eb8619a8c,boston_massachusetts,NaN,NaN,2011-11-09 20:39:29 UTC+0000
10,7d425e8d7cfb4fe7b0702fff4d6d84e7,milwaukee_wisconsin,NaN,Ph.D. Student in Political Science at Harvard ...,2011-12-26 05:00:00 UTC+0000
15,68ecc66323b8418092fdd724aaf5be94,new york_new york,Special Events,Event Hospitality & Operations Specialist,2011-12-26 05:00:00 UTC+0000
18,971937860c164896a0771bf502bad482,wilmington_vermont,NaN,NaN,2011-12-26 05:00:00 UTC+0000
19,0c9a2748560541be9fe2df0d7be88282,new york_new york,NaN,NaN,2011-12-26 05:00:00 UTC+0000
27,2a8328f6ceb548699a64293f986f3d19,washington_washington,NaN,NaN,2011-12-26 05:00:00 UTC+0000
28,8e75aca48b434c79b8b4729ef871cb50,boston_massachusetts,NaN,NaN,2011-12-26 05:00:00 UTC+0000
29,01b3e033848b41f6b7a55cefc59ba61a,san francisco_california,NaN,NaN,2011-12-26 05:00:00 UTC+0000
30,c79d4e4fd9af4ab7a7e6b6f433128476,new york_new york,"education, tech, media","media, technology, education",2011-12-26 05:00:00 UTC+0000


In [32]:
add_links(G, students, "students_id", "students_location", "location")
add_links(G, profs, "professionals_id", "professionals_location", "location")

Edges (students_id->students_location,location) were added
Edges (professionals_id->professionals_location,location) were added


In [33]:
def encode_graph(G):
    """Encode the nodes of the network into integers"""
    nodes = [(n,d.get("type",None)) for n, d in G.nodes(data=True)]
    nodes_df = pd.DataFrame(nodes, columns=["id","type"]).reset_index()
    node2idx = dict(zip(nodes_df["id"],nodes_df["index"]))
    edges = [(node2idx[src], node2idx[trg], d.get("type",None)) for src, trg, d in G.edges(data=True)]
    edges_df = pd.DataFrame(edges, columns=["src","trg","type"])
    return nodes_df, edges_df

In [34]:
print(G.number_of_nodes(), G.number_of_edges())
G.remove_nodes_from(list(nx.isolates(G)))
print(G.number_of_nodes(), G.number_of_edges())

111720 329191
107895 329191


In [35]:
nodes_df, edges_df = encode_graph(G)
len(nodes_df), len(edges_df)

(107895, 329191)

In [36]:
print(nodes_df.head())
print(nodes_df["type"].value_counts())
nodes_df.to_csv("knowledge_graph_nodes.csv", index=False)

   index                                id   type
0      0  eabbdf4029734c848a9da20779637d03  group
1      1  7080bf8dcf78463bb03e6863887fd715  group
2      2  bc6fc50a2b444efc8ec47111b290ffb8  group
3      3  37f002e8d5e442ca8e36e972eaa55882  group
4      4  52419ff84d4b47bebd0b0a6c1263c296  group
student         29451
professional    27712
question        23931
tag             14404
location         7160
school           2706
industry         2470
group              49
group_type          7
Name: type, dtype: int64


In [37]:
print(edges_df.head())
print(edges_df["type"].value_counts())
edges_df[["src","trg"]].to_csv("knowledge_graph_edges.csv", index=False, header=False, sep=" ")

   src  trg  type
0    0   49  type
1    1   49  type
2    2   49  type
3    3   49  type
4    4   49  type
follow      135907
tag          76553
location     60724
type         25625
author       23931
member        6451
Name: type, dtype: int64


In [38]:
edge_list = list(zip(edges_df["src"],edges_df["trg"]))
edge_list[:5]

[(0, 49), (1, 49), (2, 49), (3, 49), (4, 49)]

In [39]:
KG = nx.Graph(edge_list)
KG.number_of_nodes(), KG.number_of_edges()

(107895, 329191)

In [40]:
largest_cc = max(nx.connected_components(KG), key=len)
KG = nx.subgraph(KG, largest_cc)
KG.number_of_nodes(), KG.number_of_edges()

(107577, 329008)